In [128]:
# import library 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform
from support_functions import missing_value, fill_missing, list_dtypes

# sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import pickle

In [129]:
from sklearn import show_versions
show_versions()

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7ee4863007c0>
Traceback (most recent call last):
  File "/home/ruholraffen/miniconda3/envs/vertex-emen27/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ruholraffen/miniconda3/envs/vertex-emen27/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ruholraffen/miniconda3/envs/vertex-emen27/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/home/ruholraffen/miniconda3/envs/vertex-emen27/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config 


System:
    python: 3.11.7 (main, Dec 15 2023, 18:12:31) [GCC 11.2.0]
executable: /home/ruholraffen/miniconda3/envs/vertex-emen27/bin/python
   machine: Linux-6.1.85+-x86_64-with-glibc2.35

Python dependencies:
      sklearn: 1.2.2
          pip: 24.0
   setuptools: 68.2.2
        numpy: 1.26.4
        scipy: 1.11.4
       Cython: None
       pandas: 2.1.4
   matplotlib: 3.8.0
       joblib: 1.2.0
threadpoolctl: 2.2.0

Built with OpenMP: True

threadpoolctl info:
       filepath: /home/ruholraffen/miniconda3/envs/vertex-emen27/lib/python3.11/site-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
         prefix: libgomp
       user_api: openmp
   internal_api: openmp
        version: None
    num_threads: 2

       filepath: /home/ruholraffen/miniconda3/envs/vertex-emen27/lib/python3.11/site-packages/scipy.libs/libopenblasp-r0-23e5df77.3.21.dev.so
         prefix: libopenblas
       user_api: blas
   internal_api: openblas
        version: 0.3.21.dev
    num_threads: 2
threading_la

In [130]:
# Define the original data
new_test_data = {
    "Vehicle Class": ["Two-Door Car", "Four-Door Car", "Luxury SUV", "Four-Door Car", "Two-Door Car"],
    "Coverage": ["Extended", "Basic", "Basic", "Premium", "Basic"],
    "Renew Offer Type": ["Offer1", "Offer1", "Offer1", "Offer3", "Offer3"],
    "Employment Status": ["Retired", "Employed", "Retired", "Disabled", "Medical Leave"],
    "Marital Status": ["Divorced", "Married", "Married", "Married", "Married"],
    "Education": ["High School or Below", "College", "Master", "High School or Below", "High School or Below"],
    "Number of Policies": [2.0, 6.0, 2.0, 1.0, 1.0],
    "Monthly Premium Auto": [90.0, 62.0, 62.0, 116.0, 62.0],
    "Total Claim Amount": [571.479602, 114.514440, 257.248789, 556.800000, 355.820799],
    "Income": [11828.0, 44762.0, 21514.0, 17913.0, 19030.0]
}

# Convert dictionary to DataFrame
df = pd.DataFrame(new_test_data)

# Create a synthetic 'Customer Lifetime Value' for demonstration purposes
# Here we're just using a simple function to create a target variable based on some features
df['Customer Lifetime Value'] = (
    df['Monthly Premium Auto'] * df['Number of Policies'] - df['Total Claim Amount']
)

# Define the features and target
features = df.drop(columns='Customer Lifetime Value')
target = df['Customer Lifetime Value']

# List of categorical columns
categorical_cols = features.select_dtypes(include=['object']).columns

# List of numerical columns
numerical_cols = features.select_dtypes(exclude=['object']).columns

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])


## Set up authentication using services account 

In [131]:
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/sa-development.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/dti-ds-31329ac0651d.json"

 # Load data from Google Biquery to local machine
project_id = 'dti-ds'
dataset_id = 'emen_dataset_027'
table_id = 'German_Dataset'
region = 'us-central1'
bucket_name = 'emen_gcs_027'
# blob_name = 'data/data_customer_lifetime_value.csv'

In [132]:
## set up authentication using services account 
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../emen-027/sa-development.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../emen-027/dti-ds-31329ac0651d.json"

In [133]:
# Read the model from Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob('model/emen_best_random_forest_model_027.pkl')
    blob_model.download_to_filename('emen_final_model_rf.pkl')

    print ("Read model succeeded")
except:
    raise TypeError("An exception occurred")

Read model succeeded


In [134]:
# Load the model
with open('emen_final_model_rf.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

loaded_model.predict([
    [-0.83236529, -0.53418145,  2.23222059, -1.2484557, 1. ,0. , 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.],
    [-0.83236529,  0.55604222,  0.42262584, -0.07622261, 1., 0., 0., 0., 0.,1., 0., 0., 0., 1.,0.,0., 1., 0., 0., 0., 1., 0., 0., 0.,0., 0., 1., 0.]
])

array([2492.96165077, 3892.66014688])

### Load from Big Query

In [135]:
# Create BigQuery client
client = bigquery.Client(project=project_id)

# Query
query_job = client.query(f"SELECT * FROM `{dataset_id}.{table_id}`")
auto_cloud = query_job.result().to_dataframe()

# Check column names and lengths
print("auto_cloud columns:", auto_cloud.columns)
print("Number of columns in auto_cloud:", len(auto_cloud.columns))
print("df columns:", df.columns)
print("Number of columns in df:", len(df.columns))

# Adjust column assignment
if len(auto_cloud.columns) == len(df.columns):
    auto_cloud.columns = df.columns
elif len(auto_cloud.columns) < len(df.columns):
    auto_cloud.columns = df.columns[:len(auto_cloud.columns)]
else:
    auto_cloud = auto_cloud.iloc[:, :len(df.columns)]
    auto_cloud.columns = df.columns

auto_cloud

/home/ruholraffen/miniconda3/envs/vertex-emen27/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


auto_cloud columns: Index(['ID', 'LIMIT_BAL', 'MARRIAGE', 'EDUCATION', 'SEX', 'AGE', 'BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'TARGET'],
      dtype='object')
Number of columns in auto_cloud: 10
df columns: Index(['Vehicle Class', 'Coverage', 'Renew Offer Type', 'Employment Status',
       'Marital Status', 'Education', 'Number of Policies',
       'Monthly Premium Auto', 'Total Claim Amount', 'Income',
       'Customer Lifetime Value'],
      dtype='object')
Number of columns in df: 11


,Vehicle Class,Coverage,Renew Offer Type,Employment Status,Marital Status,Education,Number of Policies,Monthly Premium Auto,Total Claim Amount,Income
0,11415,50000,married,Univ,male,23,131895,132154,129237,1
1,11223,50000,married,Univ,male,24,56626,58512,61926,1
2,11451,40000,married,Univ,male,25,12961,12029,11738,1
3,2873,30000,single,Univ,male,26,28949,30319,29400,0
4,11346,30000,married,Univ,male,26,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
16620,16241,370000,single,HighSch,female,67,258212,232225,237297,1
16621,15850,250000,married,HighSch,female,67,-200,-200,-200,1
16622,16146,280000,single,HighSch,female,67,7904,7628,10270,1
16623,15626,170000,single,HighSch,female,68,49316,49061,47209,1


### Cloud Data Prediction

In [149]:
def align_features(df, expected_feature_count):
    current_feature_count = df.shape[1]
    
    if current_feature_count < expected_feature_count:
        # Add missing columns with zeros
        for i in range(current_feature_count, expected_feature_count):
            df[f'unknown_feature_{i}'] = 0
    elif current_feature_count > expected_feature_count:
        # Remove extra columns
        df = df.iloc[:, :expected_feature_count]
    
    return df

# Preprocess data (using the function from the previous message)
df_processed = preprocess_data(df)

# Align features
df_aligned = align_features(df_processed, expected_feature_count=28)

# # Now try to make predictions
# y_pred_cloud_new_data = loaded_model.predict(df_aligned)
# y_pred_cloud_new_data

#load the model
with open('emen_final_model_rf.pkl','rb') as f :
    loaded_model=pickle.load(f)

y_pred_cloud_new_data = loaded_model.predict(df_aligned)
y_pred_cloud_new_data

/home/ruholraffen/miniconda3/envs/vertex-emen27/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/ruholraffen/miniconda3/envs/vertex-emen27/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/ruholraffen/miniconda3/envs/vertex-emen27/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/ruholraffen/miniconda3/envs/vertex-emen27/lib/python3.11/site-packages/sk

array([10188.84609153, 12116.94365785, 10109.36293949,  8555.06947907,
        8557.02824146])

In [150]:
new_test_cloud=df_aligned.copy()
new_test_cloud['Prediction CLV']=y_pred_cloud_new_data
new_test_cloud

,Vehicle Class,Number of Policies,Monthly Premium Auto,Total Claim Amount,Income,Customer Lifetime Value,Coverage_Basic,Coverage_Extended,Coverage_Premium,Renew Offer Type_Offer1,...,Education_Master,unknown_feature_20,unknown_feature_21,unknown_feature_22,unknown_feature_23,unknown_feature_24,unknown_feature_25,unknown_feature_26,unknown_feature_27,Prediction CLV
0,0.0,2.0,90.0,571.479602,11828.0,-391.479602,0.0,1.0,0.0,1.0,...,0.0,0,0,0,0,0,0,0,0,10188.846092
1,0.0,6.0,62.0,114.514440,44762.0,257.485560,1.0,0.0,0.0,1.0,...,0.0,0,0,0,0,0,0,0,0,12116.943658
2,0.0,2.0,62.0,257.248789,21514.0,-133.248789,1.0,0.0,0.0,1.0,...,1.0,0,0,0,0,0,0,0,0,10109.362939
3,0.0,1.0,116.0,556.800000,17913.0,-440.800000,0.0,0.0,1.0,0.0,...,0.0,0,0,0,0,0,0,0,0,8555.069479
4,0.0,1.0,62.0,355.820799,19030.0,-293.820799,1.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,8557.028241


### Sent to BigQuery (Dataset Creation) - Skipped use auto for sent back the prediction

In [153]:
table_id='unseen_car_clv_predicted'

# construct a BQ client object
client = bigquery.Client()

# define full table id
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

auto_cloud.columns = ['_'.join(i.split(' '))for i in auto_cloud.columns]

# Load the dataframe into the BQ table
job=client.load_table_from_dataframe(auto_cloud, table_full_id)

job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}")

Loaded 16625 rows into dti-ds.emen_dataset_027.unseen_car_clv_predicted
